##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Rock, Paper & Scissors with TensorFlow Hub - TFLite

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20Deployment/Course%202%20-%20TensorFlow%20Lite/Week%202/Exercise/TFLite_Week2_Exercise_Answer.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/lmoroney/dlaicourse/blob/master/TensorFlow%20Deployment/Course%202%20-%20TensorFlow%20Lite/Week%202/Exercise/TFLite_Week2_Exercise_Answer.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
</table>

## Setup

In [ ]:
try:
    %tensorflow_version 2.x
except:
    pass
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
from datetime import datetime
import io
import itertools
from packaging import version
from six.moves import range
import sklearn.metrics


import numpy as np
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tqdm import tqdm

print("\u2022 Using TensorFlow Version:", tf.__version__)
print("\u2022 Using TensorFlow Hub Version: ", hub.__version__)
print('\u2022 GPU Device Found.' if tf.test.is_gpu_available() else '\u2022 GPU Device Not Found. Running on CPU')

## Select the Hub/TF2 Module to Use

Hub modules for TF 1.x won't work here, please use one of the selections provided.

In [ ]:
module_selection = ("efficientnet/b4", 380, 1280) #@param ["(\"mobilenet_v2\", 224, 1280)", "(\"inception_v3\", 299, 2048)" ,"(\"efficientnet/b4\", 380, 1280)"] {type:"raw", allow-input: true}
handle_base, pixels, FV_SIZE = module_selection
MODULE_HANDLE ="https://tfhub.dev/tensorflow/efficientnet/b4/feature-vector/1"
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {} and output dimension {}".format(MODULE_HANDLE, IMAGE_SIZE, FV_SIZE))

## Data Preprocessing

Use [TensorFlow Datasets](http://tensorflow.org/datasets) to load the cats and dogs dataset.

This `tfds` package is the easiest way to load pre-defined data. If you have your own data, and are interested in importing using it with TensorFlow see [loading image data](../load_data/images.ipynb)


In [ ]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

The `tfds.load` method downloads and caches the data, and returns a `tf.data.Dataset` object. These objects provide powerful, efficient methods for manipulating data and piping it into your model.

Since `"cats_vs_dog"` doesn't define standard splits, use the subsplit feature to divide it into (train, validation, test) with 80%, 10%, 10% of the data respectively.

In [ ]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
#splits = tfds.Split.ALL.subsplit(weighted=(80, 10, 10))

splits, info = tfds.load('cassava', with_info=True, as_supervised=True, split = ['train', 'validation' ,'test' ])

(train_examples, validation_examples, test_examples) = splits





In [ ]:
num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes




print(num_classes)

In [ ]:
class_names = np.array(info.features['label'].names)
print(class_names)

In [ ]:
get_label_name = info.features['label'].int2str
#for i in range(num_classes):
 #print(get_label_name(i))

for image, label in train_examples.take(3):
  print(image.shape)
  plt.figure()
  plt.imshow(image)
  print(label)
  plt.title(get_label_name(label))

In [ ]:
train_label = [ label for image, label in train_examples]
#print(train_images)

# Clear out prior logging data.
!rm -rf logs/plots

logdir = "logs/plots/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir)

def plot_to_image(figure):
  """Converts the matplotlib plot specified by 'figure' to a PNG image and
  returns it. The supplied figure is closed and inaccessible after this call."""
  # Save the plot to a PNG in memory.
  buf = io.BytesIO()
  plt.savefig(buf, format='png')
  # Closing the figure prevents it from being displayed directly inside
  # the notebook.
  plt.close(figure)
  buf.seek(0)
  # Convert PNG buffer to TF image
  image = tf.image.decode_png(buf.getvalue(), channels=4)
  # Add the batch dimension
  image = tf.expand_dims(image, 0)
  return image

def image_grid():
  """Return a 5x5 grid of the MNIST images as a matplotlib figure."""
  # Create a figure to contain the plot.
  figure = plt.figure(figsize=(10,10))
  for i in range(25):
    # Start next subplot.
    plt.subplot(5, 5, i + 1, title=class_names[train_label[i]])
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow([image for image, label in train_examples.take(28)][i]) #cmap=plt.cm.binary)
  
  return figure

# Prepare the plot
figure = image_grid()
# Convert to image and log
with file_writer.as_default():
  tf.summary.image("Training data", plot_to_image(figure), step=0)

#

In [ ]:
%tensorboard --logdir logs/plots

### Format the Data

Use the `tf.image` module to format the images for the task.

Resize the images to a fixes input size, and rescale the input channels

In [ ]:
def format_image(image, label):
    image = tf.cast(image, tf.float32)
    #image = tf.image.grayscale_to_rgb(image)
    image = tf.image.resize(image, IMAGE_SIZE)/ 255.0
    #image = tf.keras.layers.experimental.preprocessing.RandomRotation(factor, fill_mode='reflect', interpolation='bilinear')
    print(image.get_shape)
    return  image, label

Now shuffle and batch the data


In [ ]:
BATCH_SIZE =  32#@param {type:"integer"}

In [ ]:
train_batches = train_examples.shuffle(1000).map(format_image).batch(BATCH_SIZE).prefetch(1)


validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)

cm_validation_batches = validation_examples.map(format_image).batch(2160).prefetch(1)


test_batches = test_examples.batch(1).map(format_image)

In [ ]:
from collections import Counter
import pandas  as pd

counts =[]

for _ , train_labels in train_batches.take(1414):
  counts.extend(train_labels.numpy())
a = dict(Counter(counts))

In [ ]:
print(a)


In [ ]:



df_dis = pd.DataFrame.from_dict(a , orient='index')
df_dis.plot.barh(figsize = (10,7) , legend =False, colormap='Paired' )

print(class_names)


In [ ]:
class_weight = {0: 5.7 ,
1: 1.8,
2: 3.4,
3: 1.0,
4: 8.4
}

# deal with the imbalaced dataset

Inspect a batch

In [ ]:
for image_batch, label_batch in train_batches.take(1):
    pass

image_batch.shape

In [ ]:

a,  = cm_validation_batches.take(1)
im , l = a[0], a[1]




print(im.shape)

In [ ]:
from matplotlib.colors import Normalize
import matplotlib.cm  as cm


def class_distribution(train_examples , validation_examples , test_examples):
        train_label_plot = [ label for image, label in train_examples]
        valid_label_plot = [ label for image, label in validation_examples]
        test_label_plot = [ label for image, label in test_examples]
        unique, counts = np.unique(valid_label_plot, return_counts=True)
        #print(unique)

        my_cmap = cm.get_cmap('jet')
        
        # Get normalize function (takes data in range [vmin, vmax] -> [0, 1])
        plt.figure(figsize=(5, 5))
        my_norm = Normalize(vmin=0, vmax=5)
        plt.bar(  unique, counts,color=my_cmap(my_norm(unique)))
        #plt.barh (plot the horizontal bar)

        plt.xticks(unique, class_names)
        plt.title('Class Frequency')
        plt.xlabel('Class')
        plt.ylabel('Frequency')
        plt.show()

        fig = plt.gcf

        return fig

class_distribution(train_examples , validation_examples , test_examples)

## Defining the Model

All it takes is to put a linear classifier on top of the `feature_extractor_layer` with the Hub module.

For speed, we start out with a non-trainable `feature_extractor_layer`, but you can also enable fine-tuning for greater accuracy.

In [ ]:
do_fine_tuning = True #@param {type:"boolean"}

In [ ]:
feature_extractor = hub.KerasLayer(MODULE_HANDLE,
                                   input_shape=IMAGE_SIZE + (3,), 
                                   #output_shape=[FV_SIZE],
                                   trainable=do_fine_tuning)

In [ ]:
print("Building model with", MODULE_HANDLE)

model = tf.keras.Sequential([               
        feature_extractor,
        tf.keras.layers.Dense(20, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.summary()

In [ ]:
#@title (Optional) Unfreeze some layers
NUM_LAYERS = 1 #@param {type:"slider", min:1, max:50, step:1}
      
if do_fine_tuning:
    feature_extractor.trainable = False
    #feature_extractor.trainable = True
    # fine tunning from the beginning with the pre-trained weights
    
    for layer in model.layers[-NUM_LAYERS:]:
        layer.trainable = True

else:
    feature_extractor.trainable = False

model.summary()

## Training the Model

In [ ]:
if do_fine_tuning:
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])
else:
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
!rm -rf logs/image

logdir = "logs/image/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Define the basic TensorBoard callback.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')
file_writer_roc = tf.summary.create_file_writer(logdir + '/roc')

In [ ]:
def plot_confusion_matrix(cm, class_names):
  """
  Returns a matplotlib figure containing the plotted confusion matrix.

  Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
  """
  figure = plt.figure(figsize=(8, 8))
  plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
  plt.title("Confusion matrix")
  plt.colorbar()
  tick_marks = np.arange(len(class_names))
  plt.xticks(tick_marks, class_names, rotation=45)
  plt.yticks(tick_marks, class_names)

  # Normalize the confusion matrix.
  cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

  # Use white text if squares are dark; otherwise black.
  threshold = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    color = "white" if cm[i, j] > threshold else "black"
    plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  return figure

In [ ]:

def log_confusion_matrix(epoch, logs):
  # Use the model to predict the values from the validation dataset.
  test_pred_raw = model.predict(im)
  test_pred = np.argmax(test_pred_raw, axis=1)

      # Calculate the confusion matrix.
  cm = sklearn.metrics.confusion_matrix(l, test_pred)
  # Log the confusion matrix as an image summary.
  figure = plot_confusion_matrix(cm, class_names=class_names)
  cm_image = plot_to_image(figure)

  # Log the confusion matrix as an image summary.
  with file_writer_cm.as_default():
    tf.summary.image("Confusion Matrix", cm_image, step=epoch)

# Define the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:
!pip install scikit-plot

In [ ]:
import scikitplot as skplt

def plot_roc(y_true, y_probas):
  #figure = plt.figure(figsize=(8, 8))
  figure, axes = plt.subplots(1,1, figsize = (8,8))

  skplt.metrics.plot_roc_curve(y_true, y_probas , ax =axes ,text_fontsize ='small', figsize= (8 ,8))
  plt.title('ROC')
  #plt.colorbar()
  plt.ylabel('True Label')
  plt.xlabel('Predicated Label')
  fig = plt.gcf()
  #plt.show()
  fig.savefig("test_rasterization.png", dpi=150)
  #print(fig)
  

  return fig

In [ ]:
def log_roc(epoch, logs):
  test_pred_raw = model.predict(im)
  test_pred = np.argmax(test_pred_raw, axis=1)
  #print(test_pred.shape)
  #print(l.shape)
  figure_roc = plot_roc(l, test_pred_raw)
  roc_image = plot_to_image(figure_roc)


  # Log the roc  as an image summary.
  with file_writer_roc.as_default():
    tf.summary.image("ROC", roc_image, step=epoch)

roc_callback = tf.keras.callbacks.LambdaCallback( on_epoch_end=log_roc)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.0001)


In [ ]:
EPOCHS = 60
# Start TensorBoard.


#%tensorboard --logdir logs



hist = model.fit(train_batches,
                 validation_data=validation_batches,
                 #validation_steps = 10,
                 epochs=EPOCHS,
                 callbacks=[tensorboard_callback,reduce_lr, early_stopping ]
                 )

In [ ]:
from matplotlib import pyplot as plt


fig, axs = plt.subplots(2, 1, figsize = (10, 5))

axs[0].plot(hist.history['loss'])
axs[0].plot(hist.history['val_loss'])
axs[1].plot(hist.history['accuracy'])
axs[1].plot(hist.history['val_accuracy'])

In [ ]:
predictions = []
real_label  = []
for image_batch , labels_batch in test_batches.take(3000):
  
  predictions.extend(np.argmax(model.predict(image_batch), axis =-1))
  real_label.extend(labels_batch.numpy())

In [ ]:
from sklearn.metrics import classification_report
cl_report = classification_report(predictions, real_label, target_names = list(class_names))
print("The classification report : \n", cl_report)

In [ ]:
import seaborn as sns
import pandas as pd


plt.figure(figsize = (10, 10))
cl_report = classification_report(real_label, predictions, target_names = list(class_names), output_dict = True)
sns.heatmap(pd.DataFrame(cl_report).iloc[:-1, :].T, annot = True, linewidths = .5)
print("List of the class name: \n", list(class_names))

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(real_label, predictions)
print(cm)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import itertools


def plot_confusion_matrix(cm,
                          target_names,
                          title = "Confusion Matrix",
                          cmap = None,
                          normalize = True
    
):


  accuracy = np.trace(cm) / np.sum(cm).astype('float')
  misclass = 1 - accuracy

  if cmap is None:
    cmap = plt.get_cmap('Blues')

  plt.figure(figsize = (25, 25))
  plt.imshow(cm, interpolation = 'nearest', cmap = cmap)
  plt.title(title)

  if target_names is not None:
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation = 90)
    plt.yticks(tick_marks, target_names)

  if normalize:
    cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]

  thresh = cm.max() / 1.5 if normalize else cm.max() / 2
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      if normalize:
          plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                    horizontalalignment="center",
                    color="white" if cm[i, j] > thresh else "black")
      else:
          plt.text(j, i, "{:,}".format(cm[i, j]),
                    horizontalalignment="center",
                    color="white" if cm[i, j] > thresh else "black")


  plt.tight_layout()
  plt.ylabel("True label")
  plt.xlabel('Predicted label \n accuracy = {:0.4f}; ,isclass = {:0.4f}'.format(accuracy, misclass))
  plt.show()

In [ ]:
plot_confusion_matrix(cm = cm,
                      target_names = list(class_names),
                      title = 'Confusion Matrix',
                      cmap = 'inferno_r',
                      normalize = False)